In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(rc={'figure.figsize':(20,20),'font.family':'Malgun Gothic'})

import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("201901-202003.csv")
data4 = pd.read_csv("202004.csv")

data = pd.concat([data,data4])

In [3]:
def grap_year(data):
    data = str(data)
    return int(data[:4])

def grap_month(data):
    data = str(data)
    return int(data[4:])

data = data.fillna('')
data['year'] = data['REG_YYMM'].apply(lambda x: grap_year(x))
data['month'] = data['REG_YYMM'].apply(lambda x: grap_month(x))
data = data.drop(['REG_YYMM'], axis=1)

In [4]:
# 데이터 정제
df = data.copy()
df = df.drop(['CARD_CCG_NM', 'HOM_CCG_NM'], axis=1)

columns = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'year', 'month']
df = df.groupby(columns).sum().reset_index(drop=False)

In [10]:
df1 = df.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1)

CARD_SIDO_NMs = df['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df['STD_CLSS_NM'].unique()
AGEs          = df['AGE'].unique()
SEX_CTGO_CDs  = df['SEX_CTGO_CD'].unique()
FLCs          = df['FLC'].unique()
years         = df['year'].unique()
months        = df['month'].unique()

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for AGE in AGEs:
            for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                    for year in years:
                        for month in months:
                            if year == 2020:
                                if month == 1 or month == 2 or month == 3 or month == 4:
                                    temp.append([CARD_SIDO_NM, STD_CLSS_NM, AGE, SEX_CTGO_CD, FLC, year, month])
                            else:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, AGE, SEX_CTGO_CD, FLC, year, month])
                            
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=df1.columns)

In [13]:
temp['SEX_CTGO_CD'] = temp['SEX_CTGO_CD'].astype(int)
temp['FLC'] = temp['FLC'].astype(int)
temp['year'] = temp['year'].astype(int)
temp['month'] = temp['month'].astype(int)

In [14]:
df2 = pd.merge(temp,df,how='left')
df2 = df2.fillna(0)
df2

,CARD_SIDO_NM,STD_CLSS_NM,AGE,SEX_CTGO_CD,FLC,year,month,CSTMR_CNT,AMT,CNT
0,강원,건강보조식품 소매업,20s,1,1,2019,1,4.0,311200.0,4.0
1,강원,건강보조식품 소매업,20s,1,1,2019,2,3.0,605000.0,3.0
2,강원,건강보조식품 소매업,20s,1,1,2019,6,3.0,139000.0,3.0
3,강원,건강보조식품 소매업,20s,1,1,2019,7,3.0,21000.0,3.0
4,강원,건강보조식품 소매업,20s,1,1,2019,8,12.0,95000.0,12.0
...,...,...,...,...,...,...,...,...,...,...
780635,충북,정기 항공 운송업,10s,2,5,2019,10,0.0,0.0,0.0
780636,충북,정기 항공 운송업,10s,2,5,2020,1,0.0,0.0,0.0
780637,충북,정기 항공 운송업,10s,2,5,2020,2,0.0,0.0,0.0
780638,충북,정기 항공 운송업,10s,2,5,2020,3,0.0,0.0,0.0


In [15]:
# 인코딩
from sklearn.preprocessing import LabelEncoder
dtypes = df2.dtypes
encoders = {}
for column in df2.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df2.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df2[column])

In [16]:
# feature, target 설정
from sklearn.model_selection import train_test_split

y = df_num['AMT'].values
x = df_num.drop(['CSTMR_CNT', 'AMT', 'CNT'], axis=1).values

x_train, x_val, y_train, y_val = train_test_split(x, y, train_size=0.8, test_size=0.2)

In [17]:
import lightgbm as lgb

train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [31]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

model = lgb.train(params,
                  train_ds,
                  10000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 5.19179e+08
[200]	valid_0's rmse: 4.32111e+08
[300]	valid_0's rmse: 3.86654e+08
[400]	valid_0's rmse: 3.40389e+08
[500]	valid_0's rmse: 3.17724e+08
[600]	valid_0's rmse: 3.05014e+08
[700]	valid_0's rmse: 2.92774e+08
[800]	valid_0's rmse: 2.79304e+08
[900]	valid_0's rmse: 2.72303e+08
[1000]	valid_0's rmse: 2.64289e+08
[1100]	valid_0's rmse: 2.5691e+08
[1200]	valid_0's rmse: 2.4909e+08
[1300]	valid_0's rmse: 2.42991e+08
[1400]	valid_0's rmse: 2.36756e+08
[1500]	valid_0's rmse: 2.31165e+08
[1600]	valid_0's rmse: 2.27652e+08
[1700]	valid_0's rmse: 2.23919e+08
[1800]	valid_0's rmse: 2.20615e+08
[1900]	valid_0's rmse: 2.17509e+08
[2000]	valid_0's rmse: 2.13814e+08
[2100]	valid_0's rmse: 2.10937e+08
[2200]	valid_0's rmse: 2.07805e+08
[2300]	valid_0's rmse: 2.05984e+08
[2400]	valid_0's rmse: 2.034e+08
[2500]	valid_0's rmse: 2.00982e+08
[2600]	valid_0's rmse: 1.98179e+08
[2700]	valid_0's rmse: 1.95501e+08
[2800]

In [32]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4,7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for AGE in AGEs:
            for SEX_CTGO_CD in SEX_CTGO_CDs:
                for FLC in FLCs:
                    for year in years:
                        for month in months:
                            temp.append([CARD_SIDO_NM, STD_CLSS_NM, AGE, SEX_CTGO_CD, FLC, year, month])
temp = np.array(temp)
temp = pd.DataFrame(data=temp, columns=columns)

In [33]:
# 예측
pred = model.predict(temp)
temp['AMT'] = np.round(pred, 0)
temp['REG_YYMM'] = temp['year']*100 + temp['month']
temp = temp[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp = temp.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [34]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

In [35]:
# 제출 파일 만들기
submission = pd.read_csv('submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission2.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,1.253950e+08
1,202004,강원,골프장 운영업,4.581934e+09
2,202004,강원,과실 및 채소 소매업,1.630055e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,1.659908e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.212509e+08


In [23]:
temp['AMT']

0       9.643884e+07
1       4.544969e+09
2       1.200505e+09
3       1.704688e+07
4       4.292200e+04
            ...     
1389    1.549725e+09
1390    2.205159e+10
1391    1.356306e+07
1392    4.803797e+08
1393    5.258518e+07
Name: AMT, Length: 1394, dtype: float64